# Get count from Naver API

## Preprocessing

In [1]:
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip

     | 481kB 9.7MB/s
  Running setup.py bdist_wheel for tqdm ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-6fxpbuma/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
pymc3 3.6 has requirement joblib<0.13.0, but you'll have joblib 0.13.0 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
from tqdm import tqdm_notebook as tqdm
import json
from google.colab import drive 
import urllib.request
from pandas.io.json import json_normalize
drive.mount('/content/gdrive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
prod = pd.read_csv('gdrive/My Drive/공모전/Lpoint/data/Pruduct.csv')
mast = pd.read_csv('gdrive/My Drive/공모전/Lpoint/data/Master.csv')
# sess = pd.read_csv('gdrive/My Drive/공모전/엘포인트/Session.csv')
# sear = pd.read_csv('gdrive/My Drive/공모전/엘포인트/Search1.csv')
# cust = pd.read_csv('gdrive/My Drive/공모전/엘포인트/Custom.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### 자료형 변환
- **object** to **integer**

In [4]:
prod['PD_BUY_AM'] = pd.to_numeric(prod['PD_BUY_AM'].str.replace(',',''))
prod['PD_BUY_CT'][29649] = 1440
prod['PD_BUY_CT'][34105] = 1387
prod['PD_BUY_CT'] = pd.to_numeric(prod['PD_BUY_CT'])

# sess['TOT_SESS_HR_V'] = pd.to_numeric(sess['TOT_SESS_HR_V'].str.replace(',',''))
# sess['SESS_HR_PAG_VIEW'] = sess['TOT_SESS_HR_V'] / sess['TOT_PAG_VIEW_CT']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


#### 브랜드명 수정
- 브랜드명에서 **불필요한 단어** 제거(예. 향수, 괄호, 공백 등)
- **잘못입력된** 브랜드명 수정

In [0]:
prod['PD_BRA_NM'] = prod['PD_BRA_NM'].map(lambda x: x.split('(')[0].replace('[','').replace(']','').replace(' ',''))
prod.loc[prod['PD_BRA_NM']=='MADE IN ITALY','PD_BRA_NM'] = 'V73'
prod.loc[prod['PD_BRA_NM'].isin(['1909535','19330640825']),'PD_BRA_NM'] = 'KL'
prod.loc[prod['PD_BRA_NM']=='19386640906','PD_BRA_NM'] = '지이크'
prod.loc[prod['PD_BRA_NM'].isin(['1907517','1924113','1906469','1906471','1902295','1908178','1881326','1924116','1906492','1902287']),'PD_BRA_NM'] = 'CC collect'
prod.loc[prod['PD_BRA_NM'].isin(['19379580904','19364580830','1872892','1898551','18703070925','19359410830']),'PD_BRA_NM'] = '더아이잗'
prod.loc[prod['PD_BRA_NM'].isin(['1904418','1896342']),'PD_BRA_NM'] = '보니스팍스'
prod.loc[prod['PD_BRA_NM'].isin(['1912506','1912518']),'PD_BRA_NM'] = '리스트'
prod.loc[prod['PD_BRA_NM']=='19326280823','PD_BRA_NM'] = 'ENC'
prod.loc[prod['PD_BRA_NM'].isin(['19266920905','19266920905','19266920905']),'PD_BRA_NM'] = '요하넥스'
prod.loc[prod['PD_BRA_NM']=='1913802','PD_BRA_NM'] = '꼼빠니아'
prod.loc[prod['PD_BRA_NM']=='1917759','PD_BRA_NM'] = '인디안'

In [0]:
prod_mast = pd.merge(prod, mast, how = 'inner', on=['PD_C'])

# df[df['PD_BUY_CT']>=150000].sort_values(by=['PD_BUY_AM'],ascending=False)
cate_list= ['패션잡화','여성의류','화장품/뷰티케어','남성의류']
prod_mast = prod_mast[prod_mast['CLAC1_NM'].isin(cate_list)]

In [59]:
prod_mast.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2255174 entries, 0 to 5024905
Data columns (total 12 columns):
CLNT_ID      int64
SESS_ID      int64
HITS_SEQ     int64
PD_C         int64
PD_ADD_NM    object
PD_BRA_NM    object
PD_BUY_AM    int64
PD_BUY_CT    int64
PD_NM        object
CLAC1_NM     object
CLAC2_NM     object
CLAC3_NM     object
dtypes: int64(6), object(6)
memory usage: 223.7+ MB


In [7]:
l = ['18703070925','1872892','1881326','1896342','1898551','1902287','1902295','1904418','1906469','1906471','1906492','1907517','1908178',
     '1912506','1912518','1913802', '1917759','1924113','1924116','19266920905','19326280823','19359410830','19364580830','19379580904']
prod_mast1[prod_mast1['PD_BRA_NM'].isin(l)]

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM


## Luxury Brand
- 롯데에서 선정한 명품 브랜드

In [8]:
url_csm = 'http://www.lotte.com/display/searchBrandInitialReNew.lotte?upr_disp_no=1692074&kor_eng_sct_cd=&sch_nm=&sch_nm_fortis='

res = requests.get(url_csm).text
soup = bs(res, 'html.parser').find('div',{'class':'group'})
words_csm = []
for alpha in soup.find_all('li'):
  for word in alpha.find_all('li'):
    words_csm.extend(word.text.replace(')','').replace(' ','').split('('))
print(words_csm)

['겔랑', '꼬달리', '끌레드뽀보떼', '니콜라이', '닐스야드', '닥터자르트', '달팡', '더사가오브수', '더센트오프디파쳐', '데코르테', '델라롬', '디어달리아', '딥티크', '라메르', '라보라토리오올파티보', '라비오뜨', '라프레리', '랑방', '랑콤', '랩시리즈', '로라메르시에', '로베르트피게', '록시땅', '르꽁뚜아르뒤뱅', '르네휘테르', '리리코스', '리엔케이', '메이크업포에버', '메종프란시스커정', '멜비타', '몰튼브라운', '무코타', '미우미우', '미키모토코스메틱', '바닐라코', '바비브라운', '바이레도', '베네피트', '벨포트', '부르조아', '불가리', '블리스', '비디비치', '비오템', '빌리프', '산심', '산타마리아노벨라', '샤넬', '샹테카이', '설화수', '세르주루텐', '숨37˚', '슈에무라', '스틸라', '시세이도', '시슬리', '시에로코스메틱', '쌍빠', '씨앤피알엑스', '아닉구딸', '아모레퍼시픽', '아베다', '아이뽀', '아쿠아디파르마', '아틀리에코롱', '안나수이', '어딕션', '어반디케이', '에르바리오토스카노', '에르보리앙', '에스쁘아', '에스티로더', '에어린', '엘리자베스아덴', '오리진스', '오휘', '윈뉘어발리', '이금희피부밥', '이솝', '이오시카', '입생로랑', '정샘물뷰티', '조말론런던', 'JoMaloneLondon', '조르지오아르마니', '주디스리버', '쥴라이', '쥴리크', '질스튜어트', '향수', '츄파춥스뷰티', '크리니크', '크리드', '클라란스', '클라리소닉', '키엘', '탄', '토리버치', '톰포드', '투쿨포스쿨', '트리아뷰티', '펜할리곤스', '폴앤조', '프라나롬', '프라다', '프레데릭말', '프레쉬', '프리메라', '필로소피', '하이코스퍼퓸', '헤라', '헤어리추얼바이시슬리', '후', 'CEOParfums', 'DIOR', 'DIORBACKSTAGE', 'M.A.C', 'N

In [9]:
url_lux = 'http://www.lotte.com/display/searchBrandInitialReNew.lotte?upr_disp_no=5407978&kor_eng_sct_cd=&sch_nm=&sch_nm_fortis='

res = requests.get(url_lux).text
soup = bs(res, 'html.parser').find('div',{'class':'group'})
words_lux = []
for alpha in soup.find_all('li'):
  for word in alpha.find_all('li'):
    words_lux.append(word.text.split('(')[0].replace(' ',''))
print(words_lux)

['게스', '겐조', '노미네이션', '닥스', '더블엠', '라도', '러브캣', '러브캣비쥬', '레노마', '로이드', '로즈몽', '로즈몽', '루이까또즈', '루이까또즈', '루첸리', '리타모니카', '마리아꾸르끼', '마리앤잭', '마이클코어스', '마크바이마크제이콥스', '메트로시티', '메트로시티', '메트로시티', '몬데인', '몰리즈', '미니골드', '미쉘에블랑', '바쵸바치', '베이비지', '뷰', '브룩스브라더스', '비너스', '비비안', '비비안웨스트우드', '비체', '샤링', '순토', '스와로브스키', '스와치', '스타일러스', '스톤헨지', '시티즌', '아가타', '아가타', '아르키메데스', '아이그너', '아이그너', '아쿠아스큐텀', '에스티듀퐁', '엘르', '엠포리오아르마니', '올리비아버튼', '융한스', '제이에스티나', '지샥', '질스튜어트', '캘빈클라인', '캘빈클라인', '킨트', '토마스사보', '토마스사보', '티르리르', '티쏘', '페라가모', '프레드릭콘스탄트', '프린세스', '해밀턴', '휴고보스', 'DKNY', '아가타', '아가타', '아르키메데스', '아이그너', '아이그너', '아쿠아스큐텀', '바쵸바치', '베이비지', '브룩스브라더스', '비체', '시티즌', '캘빈클라인', '캘빈클라인', '닥스', '더블엠', 'DKNY', '엘르', '엠포리오아르마니', '페라가모', '프레드릭콘스탄트', '게스', '지샥', '해밀턴', '휴고보스', '메트로시티', '융한스', '제이에스티나', '질스튜어트', '겐조', '킨트', '러브캣', '러브캣비쥬', '루이까또즈', '루첸리', '마리아꾸르끼', '마리앤잭', '마이클코어스', '마크바이마크제이콥스', '메트로시티', '몬데인', '몰리즈', '미니골드', '미쉘에블랑', '노미네이션', '올리비아버튼', '프린세스', '라도', '레노마', '로즈몽', '로즈몽', '리타모니카', '샤링', '순토', '스와로브스키'

In [10]:
url_clc = 'http://www.lotte.com/display/searchBrandInitialReNew.lotte?upr_disp_no=5559034&kor_eng_sct_cd=&sch_nm=&sch_nm_fortis='

res = requests.get(url_clc).text
soup = bs(res, 'html.parser').find('div',{'class':'group'})
words_clc = []
for alpha in soup.find_all('li'):
  for word in alpha.find_all('li'):
    words_clc.extend(word.text.replace(')','').replace(' ','').split('('))
print(words_clc)

['겐조', 'KENZO', '골든구스', 'GOLDENGOOSE', '꼼데가르송', 'CommedesGarcons', '끌로에', 'CHLOE', '노비스', 'NOBIS', '닐바렛', 'neilbarrett', '듀베티카', 'duvetica', '디스퀘어드2', 'DSQUARED2', '레페토', 'Lepetto', '릭오웬스', 'RICKOWENS', '마르니', 'MARNI', '마크제이콥스', 'MarcJacobs', '막스마라', 'MaxMara', '멀버리', 'Mulberry', '몽블랑', 'MONTBLANC', '몽클레어', 'Moncler', '무스너클', 'MOOSEKNUCKLES', '미우미우', 'MIUMIU', '발렌시아가', 'BALENCIAGA', '발렌티노', 'VALENTINO', '발망', 'BALMAIN', '보테가베네타', 'BOTTEGAVENETA', '생로랑', 'SAINTLAURENT', '셀린느', 'CELINE', '스텔라메카트니', 'StellaMcCartney', '스톤아일랜드', 'STONEISLAND', '스튜어트와이츠먼', 'StuartWeitzman', '아크네', 'Acne', '안야힌드마치', 'ANYAHINDMARCH', '알렉산더맥퀸', 'ALEXANDERMCQUEEN', '알렉산더왕', 'ALEXANDERWANG', '에르노', 'herno', '에트로', 'ETRO', '올라카일리', 'OrlaKiely', '이자벨마랑', 'ISABELMARANT', '자넬라토', 'ZANELLATO', '지방시', 'GIVENCHY', '커먼프로젝트', 'COMMONPROJECTS', '코치', 'COACH', '키아라페라그니', 'chiaraferragni', '토리버치', 'TORYBURCH', '톰브라운', 'THOMBROWNE', '투미', 'TUMI', '폴스미스', 'Paulsmith', '프라다', 'PRADA', '프로엔자슐러', 'proenzaschouler', '피에르아르디', 'pi

In [11]:
brand_lotte = words_csm
brand_lotte.extend(words_lux)
brand_lotte.extend(words_clc)
brand_lotte = list(set(brand_lotte))
len(brand_lotte)

275

## 부록
- 네이버 API를 이용해서 브랜드 + 중분류 검색어트렌드를 카운트하는 코드

In [0]:
df2 = pd.read_csv('gdrive/My Drive/공모전/Lpoint/명품 골라내기/my_luxury_list.csv')

In [13]:
brands = brand_lotte
brands.extend(df2['title'])
brands = list(set(brands))
len(brands)

463

In [0]:
df1 = prod_mast1.groupby(['CLAC2_NM','CLAC3_NM','PD_BRA_NM']).agg({'PD_BUY_AM':'mean','PD_BUY_CT':'sum'}).reset_index()

In [0]:
df1 = df1[df1['PD_BRA_NM'].isin(brands)]

In [16]:
len(df1)

4157

In [0]:
client_id = ""
client_secret = ""
url = "https://openapi.naver.com/v1/datalab/search"

In [21]:
bra_list = []
for brand in tqdm(brands):
    a = brand
    bra_list.append({"groupName": a,
                    "keywords": [a]
                    })

len(bra_list)

463

In [46]:
bra_trend = []
lists = [bra_list[0]]
for i in tqdm(range(1,len(bra_list[:]),4)):
  b_trends = []
  lists.extend(bra_list[i:i+4])
  body = '{\"startDate\":\"2016-01-01\",\"endDate\":\"2019-01-30\",\"timeUnit\":\"month\",\"keywordGroups\":%s}'%json.dumps(lists)
  request = urllib.request.Request(url)
  request.add_header("X-Naver-Client-Id",client_id)
  request.add_header("X-Naver-Client-Secret",client_secret)
  request.add_header("Content-Type","application/json")
  response = urllib.request.urlopen(request, data=body.encode("utf-8"))
  rescode = response.getcode()
  response_body = response.read().decode('utf-8')
  trends = json.loads(response_body)['results']
  for t in trends:
    b_trends.append({
        'title':t['title'],
        'ratio':np.nanmean([d['ratio'] for d in t['data']])
    })
  bra_trend.extend(b_trends)
  max_b =  max(b_trends, key=lambda x:x['ratio'])
  lists = list(filter(lambda x:x['groupName']==max_b['title'], lists))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: Mean of empty slice


In [37]:
cate_list = []
for category in tqdm(df1['CLAC3_NM'].drop_duplicates()):
  a = category
  cate_list.append({"groupName": a,
                    "keywords": a.split('/')
                    })

len(cate_list)

174

In [41]:
cate_trend = []
lists = [cate_list[0]]
for i in tqdm(range(1,len(cate_list[:]),4)):
  c_trends = []
  lists.extend(cate_list[i:i+4])
  body = '{\"startDate\":\"2016-01-01\",\"endDate\":\"2019-01-30\",\"timeUnit\":\"month\",\"keywordGroups\":%s}'%json.dumps(lists)
  request = urllib.request.Request(url)
  request.add_header("X-Naver-Client-Id",client_id)
  request.add_header("X-Naver-Client-Secret",client_secret)
  request.add_header("Content-Type","application/json")
  response = urllib.request.urlopen(request, data=body.encode("utf-8"))
  rescode = response.getcode()
  response_body = response.read().decode('utf-8')
  trends = json.loads(response_body)['results']
  for t in trends:
    c_trends.append({
        'title':t['title'],
        'ratio':np.nanmean([d['ratio'] for d in t['data']])
    })
  cate_trend.extend(c_trends)
  max_c =  max(c_trends, key=lambda x:x['ratio'])
  lists = list(filter(lambda x:x['groupName']==max_c['title'], lists))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: Mean of empty slice


In [0]:
brand_mp = json_normalize(bra_trend).drop_duplicates().reset_index().fillna(0)

In [0]:
category_mp = json_normalize(cate_trend).drop_duplicates().reset_index().fillna(0)

In [0]:
brand_mp = brand_mp[['title','ratio']]
category_mp = category_mp[['title','ratio']]

In [53]:
df = prod_mast[prod_mast['PD_BRA_NM'].isin(brands)]
df2 = pd.merge(df,brand_mp, how = 'inner', left_on = ['PD_BRA_NM'], right_on = ['title'])
df2 = pd.merge(df2,category_mp, how = 'inner', left_on = ['CLAC3_NM'], right_on = ['title'])
# df2[[]]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 697754 entries, 0 to 697753
Data columns (total 16 columns):
CLNT_ID      697754 non-null int64
SESS_ID      697754 non-null int64
HITS_SEQ     697754 non-null int64
PD_C         697754 non-null int64
PD_ADD_NM    697754 non-null object
PD_BRA_NM    697754 non-null object
PD_BUY_AM    697754 non-null int64
PD_BUY_CT    697754 non-null int64
PD_NM        697754 non-null object
CLAC1_NM     697754 non-null object
CLAC2_NM     697754 non-null object
CLAC3_NM     697754 non-null object
title_x      697754 non-null object
ratio_x      697754 non-null float64
title_y      697754 non-null object
ratio_y      697754 non-null float64
dtypes: float64(2), int64(6), object(8)
memory usage: 90.5+ MB


In [63]:
df2.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,TREND_SC
0,4139680,7605037,12,642112,색상:워터멜론,바비브라운,39000,1,크러쉬드 립칼라 - 플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,25.959711
1,4350334,2023661,77,642112,색상:엔젤 / 1개,바비브라운,39000,1,크러쉬드 립칼라 - 플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,25.959711
2,5172356,4714565,47,642112,색상:카바나,바비브라운,39000,1,크러쉬드 립칼라 - 플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,25.959711
3,5444959,6843465,24,642112,색상:워터멜론 / 1개,바비브라운,39000,1,크러쉬드 립칼라 - 플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,25.959711
4,5461276,7896792,53,642112,색상:워터멜론 / 1개,바비브라운,39000,1,크러쉬드 립칼라 - 플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,25.959711


In [0]:
df2['TREND_SC'] = df2['ratio_x'] + df2['ratio_y']

In [0]:
colnames = ['title_x','title_y','ratio_x','ratio_y']
df2.drop(colnames, axis=1, inplace = True)

In [0]:
df2.to_csv('gdrive/My Drive/공모전/Lpoint/선호지수 만들기/trend_score.csv', index = False)

In [58]:
df2.groupby(['CLAC3_NM','PD_BRA_NM'])['TREND_SC'].mean().reset_index()

,CLAC3_NM,PD_BRA_NM,TREND_SC
0,2단우산,루이까또즈,31.624072
1,2단우산,엘르,4.156726
2,2단우산,탠디,0.422276
3,3단우산,탠디,0.474579
4,BB/파운데이션/컴팩트류,SK-II,25.999873
5,BB/파운데이션/컴팩트류,VDL,33.734991
6,BB/파운데이션/컴팩트류,VT코스메틱,29.419888
7,BB/파운데이션/컴팩트류,겔랑,29.477231
8,BB/파운데이션/컴팩트류,끌레드뽀보떼,27.975340
9,BB/파운데이션/컴팩트류,닥터자르트,34.677252
